### Install Libraries

In [1]:
! pip install sqlalchemy==1.4.46
! pip install pandasql==0.7.3
! pip install xgboost
! pip install psycopg2


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Import Libraries

In [1]:
import sys
sys.path.append("../")
from sales import data_holder, data_info, common, local_settings
from sales.feature_engineering \
    import sku_related_features, cc_related_features,\
        date_related_features, warehouse_related_features, \
            company_related_features, stock_related_features
            
import pandas as pd
import pandasql as ps
from io import StringIO
from psycopg2 import sql
import psycopg2

pd.set_option('display.max_columns', None)

### Define utils functions

##### Load dataframe in datatable

In [2]:
def load_dataframe_in_datatable(conn, create_query, table_name, df):
    buffer = StringIO()
    df.to_csv(buffer, index=False, header=False)
    print('CSV Ready')
    buffer.seek(0)
    """Create DB connection, create table and execute COPY FROM"""
    with conn.cursor() as cursor:
        cursor.execute(create_query)
        cursor.copy_from(buffer, table_name, sep=',')
        conn.commit()
    print('Data copied in datatable')
     

##### Get datatable in dataframe

In [3]:
def get_datatable_to_df(conn, table_name):
    with conn.cursor() as cursor:
        cursor.execute(f'SELECT * FROM {table_name};')
        return pd.DataFrame(
                    cursor.fetchall(),
                    columns=[desc[0] for desc in cursor.description]
                )

##### Database Connection

In [4]:
def connect():
    return psycopg2.connect(
        host=local_settings.PRODDBHOST,
        database=local_settings.PRODDBNAME,
        user=local_settings.PRODDBUSER,
        password=local_settings.PRODDBPASS
    )

### Load data in Database

#### Load Warehouse data

In [13]:
create_df_warehouse_query = """
DROP TABLE df_warehouse;
CREATE TABLE df_warehouse (
    warehouse VARCHAR(30),
    is_metropolitan_zone INT,
    is_inside_mall INT
);"""

In [16]:
conn = connect()

df_warehouses = pd.read_csv('../data/data warehouses/warehouses.csv', delimiter=';')
load_dataframe_in_datatable(conn, create_df_warehouse_query, 'df_warehouse', df_warehouses)
with conn.cursor() as cursor:
    cursor.execute('SELECT * FROM df_warehouse;')
    df = pd.DataFrame(
                cursor.fetchall(),
                columns=[desc[0] for desc in cursor.description]
        )


conn.close()
df.head()

CSV Ready


DuplicateTable: la relación «df_warehouse» ya existe


#### Load Campaigns data

In [21]:
create_conn_sale_campaigns_query = """
DROP TABLE conn_campaign;
CREATE TABLE conn_campaign (
    start_date DATE,
    end_date DATE,
    campaign_name VARCHAR(30)
);"""

In [22]:
conn = connect()

df_sale_campaigns = pd.read_csv('../data/data sale campaigns/sale_campaigns.csv', delimiter=';', encoding='latin-1')
df_sale_campaigns['start_date'] = pd.to_datetime(df_sale_campaigns['start_date'], format='%d-%m-%Y')
df_sale_campaigns['end_date'] = pd.to_datetime(df_sale_campaigns['end_date'], format='%d-%m-%Y')

load_dataframe_in_datatable(conn, create_df_sale_campaigns_query, 'df_campaign', df_sale_campaigns)
with conn.cursor() as cursor:
    cursor.execute('SELECT * FROM df_campaign;')
    df = pd.DataFrame(
                cursor.fetchall(),
                columns=[desc[0] for desc in cursor.description]
            )

conn.close()
df.head()

CSV Ready
Data copied in datatable


,start_date,end_date,campaign_name
0,2019-03-01,2019-03-17,SALE
1,2019-05-01,2019-05-12,DIA DE LA MADRE
2,2019-06-05,2019-06-16,DIA DEL PADRE
3,2019-08-30,2019-09-23,SALE
4,2020-02-21,2020-03-15,SALE


#### Load Sale data

In [40]:
create_df_sale_query = """
DROP TABLE df_sale;
CREATE TABLE df_sale (
    date DATE,
    sku VARCHAR(30),
    cc VARCHAR(30),
    itemnumber VARCHAR(30),
    warehouse VARCHAR(40),
    quantity INT,
    week_stock_quantity INT,
    unit_price INT,
    cost INT    
);"""

In [41]:
conn = connect()

df_sale = pd.read_csv('../data/filled_processed_data/filled_processed_best_sellers_data.csv')

df_sale.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1, inplace=True)
df_sale['week_stock_quantity'] = df_sale['week_stock_quantity'].fillna(0).astype(int)
df_sale['unit_price'] = df_sale['unit_price'].fillna(0).astype(int)
df_sale['cost'] = df_sale['cost'].fillna(0).astype(int)

load_dataframe_in_datatable(conn, create_df_sale_query, 'df_sale', df_sale)
with conn.cursor() as cursor:
    cursor.execute('SELECT * FROM df_sale;')
    df = pd.DataFrame(
                cursor.fetchall(),
                columns=[desc[0] for desc in cursor.description]
            )
    
conn.close()

data_holder.set_data(df)
data_info.preview()

CSV Ready
Data copied in datatable
Data shape:  (631355, 9)


,date,sku,cc,itemnumber,warehouse,quantity,week_stock_quantity,unit_price,cost
0,2020-01-02,55120DDNM26,55120DDNM,55120,BUENAVENTURA,0,1,0,0
1,2020-01-02,55181ASHT30,55181ASHT,55181,BUENAVENTURA,0,1,41176,12760
2,2020-01-02,55181ASHT31,55181ASHT,55181,BUENAVENTURA,0,1,0,0
3,2020-01-02,55181ASHT36,55181ASHT,55181,BUENAVENTURA,0,1,41176,14169
4,2020-01-02,55181FGE30,55181FGE,55181,BUENAVENTURA,0,3,41176,12346


### Feature engineering

#### Add *Date* relate information

In [42]:
conn = connect()

print("#### Adding Date related info")

date_related_features.add_weekday_information(conn)
print("Ok feature 1")
date_related_features.add_month_information(conn)
print("Ok feature 2")
date_related_features.add_year_information(conn)
print("Ok feature 3")
date_related_features.add_offer_day_information(conn)
print("Ok feature 4")


data_holder.set_data(get_datatable_to_df(conn, 'df_sale'))

conn.close()
data_info.preview()

#### Adding Date related info
Ok feature 1
Ok feature 2
Ok feature 3
Ok feature 4
Data shape:  (631355, 13)


,date,sku,cc,itemnumber,warehouse,quantity,week_stock_quantity,unit_price,cost,weekday,month,year,is_offer_day
0,2020-01-08,55416FGE0,55416FGE,55416,TEMUCO,0,1,0,0,3,1,2020,0
1,2020-01-12,55416FGE2,55416FGE,55416,PUCON,0,3,36975,11737,0,1,2020,0
2,2020-02-05,55416FGE2,55416FGE,55416,TEMUCO,0,3,32773,12047,3,2,2020,0
3,2020-02-13,55416FGE0,55416FGE,55416,TEMUCO,0,1,0,0,4,2,2020,0
4,2020-02-16,55416FGE0,55416FGE,55416,TEMUCO,0,1,0,0,0,2,2020,0


#### Add *Warehouse* relate information

In [43]:
conn = connect()

print("#### Adding Warehouse related info")

warehouse_related_features.add_warehouse_is_metropolitan_zone(conn)
print("Ok feature 1")
warehouse_related_features.add_warehouse_is_inside_mall(conn)
print("Ok feature 2")
warehouse_related_features.add_warehouse_last_xdays_sales(conn, 7)
print("Ok feature 3")
warehouse_related_features.add_warehouse_last_xdays_sales(conn, 15)
print("Ok feature 4")
warehouse_related_features.add_warehouse_last_xdays_sales(conn, 30)
print("Ok feature 5")
warehouse_related_features.add_warehouse_cumulative_sales_in_the_week(conn)
print("Ok feature 6")
warehouse_related_features.add_warehouse_cumulative_sales_in_the_month(conn)
print("Ok feature 7")
warehouse_related_features.add_warehouse_cumulative_sales_in_the_year(conn)
print("Ok feature 8")
warehouse_related_features.add_warehouse_last_xdays_mean_sales(conn, 7)
print("Ok feature 9")
warehouse_related_features.add_warehouse_last_xdays_mean_sales(conn, 15)
print("Ok feature 10")
warehouse_related_features.add_warehouse_last_xdays_mean_sales(conn, 30)

data_holder.set_data(get_datatable_to_df(conn, 'df_sale'))

conn.close()
data_info.preview()

#### Adding Warehouse related info
Ok feature 1
Ok feature 2
Ok feature 3
Ok feature 4
Ok feature 5
Ok feature 6
Ok feature 7
Ok feature 8
Ok feature 9
Ok feature 10
Data shape:  (631355, 24)


,date,sku,cc,itemnumber,warehouse,quantity,week_stock_quantity,unit_price,cost,weekday,month,year,is_offer_day,is_metropolitan_zone,is_inside_mall,warehouse_last_7days_sales,warehouse_last_15days_sales,warehouse_last_30days_sales,warehouse_cumulative_sales_in_the_week,warehouse_cumulative_sales_in_the_month,warehouse_cumulative_sales_in_the_year,warehouse_last_7days_mean_sales,warehouse_last_15days_mean_sales,warehouse_last_30days_mean_sales
0,2020-01-08,55416FGE0,55416FGE,55416,TEMUCO,0,1,0,0,3,1,2020,0,0,0,2,2,2,2,2,2,0.2857,0.1333,0.0667
1,2020-01-12,55416FGE2,55416FGE,55416,PUCON,0,3,36975,11737,0,1,2020,0,0,0,2,4,4,2,4,4,0.2857,0.2667,0.1333
2,2020-02-05,55416FGE2,55416FGE,55416,TEMUCO,0,3,32773,12047,3,2,2020,0,0,0,4,7,16,1,1,16,0.5714,0.4667,0.5333
3,2020-02-13,55416FGE0,55416FGE,55416,TEMUCO,0,1,0,0,4,2,2020,0,0,0,1,5,11,0,2,17,0.1429,0.3333,0.3667
4,2020-02-16,55416FGE0,55416FGE,55416,TEMUCO,0,1,0,0,0,2,2020,0,0,0,2,4,12,2,4,19,0.2857,0.2667,0.4000


#### Add *CC - Date* related info

In [44]:
conn = connect()
print("#### Adding CC-date related info")

cc_related_features.add_cc_cumulative_sales_in_the_week(conn)
print("Ok feature 1")
cc_related_features.add_cc_cumulative_sales_in_the_month(conn)
print("Ok feature 2")
cc_related_features.add_cc_cumulative_sales_in_the_year(conn)
print("Ok feature 3")
cc_related_features.add_cc_historic_sales(conn)
print("Ok feature 4")
cc_related_features.add_cc_historic_sales_same_day_of_the_week(conn)
print("Ok feature 5")
cc_related_features.add_cc_historic_sales_same_month(conn)
print("Ok feature 6")

data_holder.set_data(get_datatable_to_df(conn, 'df_sale'))

conn.close()
data_info.preview()

#### Adding CC-date related info
Ok feature 1
Ok feature 2
Ok feature 3
Ok feature 4
Ok feature 5


GroupingError: la columna «df_sale.date» debe aparecer en la cláusula GROUP BY o ser usada en una función de agregación
LINE 6:             EXTRACT(MONTH FROM date) AS month,
                                       ^


#### Add *CC-Warehouse-Date* related info

In [7]:
conn = connect()
print("#### Adding CC-warehouse-date related info")

cc_related_features.add_cc_warehouse_last_xdays_sales(conn, 7)
print("Ok feature 1")
cc_related_features.add_cc_warehouse_last_xdays_mean_sales(conn, 7)
print("Ok feature 2")
cc_related_features.add_cc_warehouse_last_xdays_sales(conn, 10)
print("Ok feature 3")
cc_related_features.add_cc_warehouse_last_xdays_mean_sales(conn, 10)
print("Ok feature 4")
cc_related_features.add_cc_warehouse_last_xdays_sales(conn, 14)
print("Ok feature 5")
cc_related_features.add_cc_warehouse_last_xdays_mean_sales(conn, 14)
print("Ok feature 6")
cc_related_features.add_cc_warehouse_last_xdays_sales(conn, 30)
print("Ok feature 7")
cc_related_features.add_cc_warehouse_last_xdays_mean_sales(conn, 30)
print("Ok feature 8")

cc_related_features.add_cc_warehouse_historic_sales(conn)
print("Ok feature 9")
cc_related_features.add_cc_warehouse_historic_sales_same_day_of_the_week(conn)
print("Ok feature 10")
cc_related_features.add_cc_warehouse_historic_sales_same_month(conn)
print("Ok feature 11")

cc_related_features.add_cc_warehouse_cumulative_sales_in_the_week(conn)
print("Ok feature 12")
cc_related_features.add_cc_warehouse_cumulative_sales_in_the_month(conn)
print("Ok feature 13")
cc_related_features.add_cc_warehouse_cumulative_sales_in_the_year(conn)
print("Ok feature 14")

cc_related_features.add_y_cc_warehouse_next_xdays_sales(conn, 7)
print("Ok feature 15")
cc_related_features.add_y_cc_warehouse_next_xdays_sales(conn, 14)
print("Ok feature 16")
cc_related_features.add_y_cc_warehouse_next_xdays_sales(conn, 30)
print("Ok feature 17")

data_holder.set_data(get_datatable_to_df(conn, 'df_sale'))

conn.close()
data_info.preview()

#### Adding CC-warehouse-date related info
Ok feature 1
Ok feature 2
Ok feature 3
Ok feature 4
Ok feature 5
Ok feature 6
Ok feature 7
Ok feature 8
Ok feature 9
Ok feature 10
Ok feature 11
Ok feature 12
Ok feature 13
Ok feature 14
Ok feature 15
Ok feature 16
Ok feature 17
Data shape:  (631355, 47)


,date,sku,cc,itemnumber,warehouse,quantity,week_stock_quantity,unit_price,cost,weekday,month,year,is_offer_day,is_metropolitan_zone,is_inside_mall,warehouse_last_7days_sales,warehouse_last_15days_sales,warehouse_last_30days_sales,warehouse_cumulative_sales_in_the_week,warehouse_cumulative_sales_in_the_month,warehouse_cumulative_sales_in_the_year,warehouse_last_7days_mean_sales,warehouse_last_15days_mean_sales,warehouse_last_30days_mean_sales,cc_cumulative_sales_in_the_week,cc_cumulative_sales_in_the_month,cc_cumulative_sales_in_the_year,cc_historic_sales,cc_historic_sales_same_day_of_the_week,cc_historic_sales_same_month,cc_warehouse_last_7days_sales,cc_warehouse_last_7days_mean_sales,cc_warehouse_last_10days_sales,cc_warehouse_last_10days_mean_sales,cc_warehouse_last_14days_sales,cc_warehouse_last_14days_mean_sales,cc_warehouse_last_30days_sales,cc_warehouse_last_30days_mean_sales,cc_warehouse_historic_sales,cc_warehouse_historic_sales_same_day_of_the_week,cc_warehouse_historic_sales_same_month,cc_warehouse_cumulative_sales_in_the_week,cc_warehouse_cumulative_sales_in_the_month,cc_warehouse_cumulative_sales_in_the_year,y_cc_warehouse_next_7days_sales,y_cc_warehouse_next_14days_sales,y_cc_warehouse_next_30days_sales
0,2022-03-23,55120DDNM30,55120DDNM,55120,ALERCE,0,0,0,0,3,3,2022,0,0,1,1,3,3,0,3,14,0.1429,0.2000,0.1,1,7,21,264,37,33,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0
1,2022-03-23,55120DDNM26,55120DDNM,55120,ALERCE,0,0,0,0,3,3,2022,0,0,1,1,3,3,0,3,14,0.1429,0.2000,0.1,1,7,21,264,37,33,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0
2,2022-03-23,55120DDNM25,55120DDNM,55120,ALERCE,0,0,0,0,3,3,2022,0,0,1,1,3,3,0,3,14,0.1429,0.2000,0.1,1,7,21,264,37,33,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0
3,2022-03-24,55120DDNM31,55120DDNM,55120,ALERCE,0,0,0,0,4,3,2022,0,0,1,1,2,3,0,3,14,0.1429,0.1333,0.1,2,8,22,265,48,33,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0
4,2022-03-24,55120DDNM24,55120DDNM,55120,ALERCE,0,0,0,0,4,3,2022,0,0,1,1,2,3,0,3,14,0.1429,0.1333,0.1,2,8,22,265,48,33,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0


#### Add *SKU - Date*  related info

In [8]:
conn = connect()
print("#### Adding Sku-date related info")

sku_related_features.add_sku_cumulative_sales_in_the_week(conn)
print("Ok feature 1")
sku_related_features.add_sku_cumulative_sales_in_the_month(conn)
print("Ok feature 2")
sku_related_features.add_sku_cumulative_sales_in_the_year(conn)
print("Ok feature 3")
sku_related_features.add_sku_historic_sales(conn)
print("Ok feature 4")
sku_related_features.add_sku_historic_sales_same_day_of_the_week(conn)
print("Ok feature 5")
sku_related_features.add_sku_historic_sales_same_month(conn)
print("Ok feature 6")

data_holder.set_data(get_datatable_to_df(conn, 'df_sale'))

conn.close()
data_info.preview()

#### Adding Sku-date related info
Ok feature 1
Ok feature 2
Ok feature 3
Ok feature 4
Ok feature 5
Ok feature 6
Data shape:  (631355, 53)


,date,sku,cc,itemnumber,warehouse,quantity,week_stock_quantity,unit_price,cost,weekday,month,year,is_offer_day,is_metropolitan_zone,is_inside_mall,warehouse_last_7days_sales,warehouse_last_15days_sales,warehouse_last_30days_sales,warehouse_cumulative_sales_in_the_week,warehouse_cumulative_sales_in_the_month,warehouse_cumulative_sales_in_the_year,warehouse_last_7days_mean_sales,warehouse_last_15days_mean_sales,warehouse_last_30days_mean_sales,cc_cumulative_sales_in_the_week,cc_cumulative_sales_in_the_month,cc_cumulative_sales_in_the_year,cc_historic_sales,cc_historic_sales_same_day_of_the_week,cc_historic_sales_same_month,cc_warehouse_last_7days_sales,cc_warehouse_last_7days_mean_sales,cc_warehouse_last_10days_sales,cc_warehouse_last_10days_mean_sales,cc_warehouse_last_14days_sales,cc_warehouse_last_14days_mean_sales,cc_warehouse_last_30days_sales,cc_warehouse_last_30days_mean_sales,cc_warehouse_historic_sales,cc_warehouse_historic_sales_same_day_of_the_week,cc_warehouse_historic_sales_same_month,cc_warehouse_cumulative_sales_in_the_week,cc_warehouse_cumulative_sales_in_the_month,cc_warehouse_cumulative_sales_in_the_year,y_cc_warehouse_next_7days_sales,y_cc_warehouse_next_14days_sales,y_cc_warehouse_next_30days_sales,sku_cumulative_sales_in_the_week,sku_cumulative_sales_in_the_month,sku_cumulative_sales_in_the_year,sku_historic_sales,sku_historic_sales_same_day_of_the_week,sku_historic_sales_same_month
0,2022-03-23,55120DDNM25,55120DDNM,55120,ALERCE,0,0,0,0,3,3,2022,0,0,1,1,3,3,0,3,14,0.1429,0.2000,0.1,1,7,21,264,37,33,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,13,3,1
1,2022-03-23,55120DDNM26,55120DDNM,55120,ALERCE,0,0,0,0,3,3,2022,0,0,1,1,3,3,0,3,14,0.1429,0.2000,0.1,1,7,21,264,37,33,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,2,26,3,4
2,2022-03-23,55120DDNM30,55120DDNM,55120,ALERCE,0,0,0,0,3,3,2022,0,0,1,1,3,3,0,3,14,0.1429,0.2000,0.1,1,7,21,264,37,33,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,1,2,41,4,5
3,2022-03-24,55120DDNM24,55120DDNM,55120,ALERCE,0,0,0,0,4,3,2022,0,0,1,1,2,3,0,3,14,0.1429,0.1333,0.1,2,8,22,265,48,33,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,1,3,3,7,4,1
4,2022-03-24,55120DDNM31,55120DDNM,55120,ALERCE,0,0,0,0,4,3,2022,0,0,1,1,2,3,0,3,14,0.1429,0.1333,0.1,2,8,22,265,48,33,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,17,5,2


#### Add *SKU-Warehouse-Date* related info

In [9]:
conn = connect()
print("#### Adding Sku-warehouse-date related info")

sku_related_features.add_sku_warehouse_last_xdays_sales(conn, 7)
print("Ok feature 1")
sku_related_features.add_sku_warehouse_last_xdays_mean_sales(conn, 7)
print("Ok feature 2")
sku_related_features.add_sku_warehouse_last_xdays_sales(conn, 10)
print("Ok feature 3")
sku_related_features.add_sku_warehouse_last_xdays_mean_sales(conn, 10)
print("Ok feature 4")
sku_related_features.add_sku_warehouse_last_xdays_sales(conn, 14)
print("Ok feature 5")
sku_related_features.add_sku_warehouse_last_xdays_mean_sales(conn, 14)
print("Ok feature 6")
sku_related_features.add_sku_warehouse_last_xdays_sales(conn, 30)
print("Ok feature 7")
sku_related_features.add_sku_warehouse_last_xdays_mean_sales(conn, 30)
print("Ok feature 8")

sku_related_features.add_sku_warehouse_historic_sales(conn)
print("Ok feature 9")
sku_related_features.add_sku_warehouse_historic_sales_same_day_of_the_week(conn)
print("Ok feature 10")
sku_related_features.add_sku_warehouse_historic_sales_same_month(conn)
print("Ok feature 11")

sku_related_features.add_sku_warehouse_cumulative_sales_in_the_week(conn)
print("Ok feature 12")
sku_related_features.add_sku_warehouse_cumulative_sales_in_the_month(conn)
print("Ok feature 13")
sku_related_features.add_sku_warehouse_cumulative_sales_in_the_year(conn)
print("Ok feature 14")

sku_related_features.add_y_sku_warehouse_next_xdays_sales(conn, 7)
print("Ok feature 15")
sku_related_features.add_y_sku_warehouse_next_xdays_sales(conn, 14)
print("Ok feature 16")
sku_related_features.add_y_sku_warehouse_next_xdays_sales(conn, 30)
print("Ok feature 17")


data_holder.set_data(get_datatable_to_df(conn, 'df_sale'))

conn.close()
data_info.preview()

#### Adding Sku-warehouse-date related info
Ok feature 1
Ok feature 2
Ok feature 3
Ok feature 4
Ok feature 5
Ok feature 6
Ok feature 7
Ok feature 8
Ok feature 9
Ok feature 10
Ok feature 11
Ok feature 12
Ok feature 13
Ok feature 14
Ok feature 15
Ok feature 16
Ok feature 17
Data shape:  (631355, 70)


,date,sku,cc,itemnumber,warehouse,quantity,week_stock_quantity,unit_price,cost,weekday,month,year,is_offer_day,is_metropolitan_zone,is_inside_mall,warehouse_last_7days_sales,warehouse_last_15days_sales,warehouse_last_30days_sales,warehouse_cumulative_sales_in_the_week,warehouse_cumulative_sales_in_the_month,warehouse_cumulative_sales_in_the_year,warehouse_last_7days_mean_sales,warehouse_last_15days_mean_sales,warehouse_last_30days_mean_sales,cc_cumulative_sales_in_the_week,cc_cumulative_sales_in_the_month,cc_cumulative_sales_in_the_year,cc_historic_sales,cc_historic_sales_same_day_of_the_week,cc_historic_sales_same_month,cc_warehouse_last_7days_sales,cc_warehouse_last_7days_mean_sales,cc_warehouse_last_10days_sales,cc_warehouse_last_10days_mean_sales,cc_warehouse_last_14days_sales,cc_warehouse_last_14days_mean_sales,cc_warehouse_last_30days_sales,cc_warehouse_last_30days_mean_sales,cc_warehouse_historic_sales,cc_warehouse_historic_sales_same_day_of_the_week,cc_warehouse_historic_sales_same_month,cc_warehouse_cumulative_sales_in_the_week,cc_warehouse_cumulative_sales_in_the_month,cc_warehouse_cumulative_sales_in_the_year,y_cc_warehouse_next_7days_sales,y_cc_warehouse_next_14days_sales,y_cc_warehouse_next_30days_sales,sku_cumulative_sales_in_the_week,sku_cumulative_sales_in_the_month,sku_cumulative_sales_in_the_year,sku_historic_sales,sku_historic_sales_same_day_of_the_week,sku_historic_sales_same_month,sku_warehouse_last_7days_sales,sku_warehouse_last_7days_mean_sales,sku_warehouse_last_10days_sales,sku_warehouse_last_10days_mean_sales,sku_warehouse_last_14days_sales,sku_warehouse_last_14days_mean_sales,sku_warehouse_last_30days_sales,sku_warehouse_last_30days_mean_sales,sku_warehouse_historic_sales,sku_warehouse_historic_sales_same_day_of_the_week,sku_warehouse_historic_sales_same_month,sku_warehouse_cumulative_sales_in_the_week,sku_warehouse_cumulative_sales_in_the_month,sku_warehouse_cumulative_sales_in_the_year,y_sku_warehouse_next_7days_sales,y_sku_warehouse_next_14days_sales,y_sku_warehouse_next_30days_sales
0,2022-03-23,55120DDNM25,55120DDNM,55120,ALERCE,0,0,0,0,3,3,2022,0,0,1,1,3,3,0,3,14,0.1429,0.2000,0.1,1,7,21,264,37,33,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,13,3,1,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0
1,2022-03-23,55120DDNM26,55120DDNM,55120,ALERCE,0,0,0,0,3,3,2022,0,0,1,1,3,3,0,3,14,0.1429,0.2000,0.1,1,7,21,264,37,33,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,2,26,3,4,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0
2,2022-03-23,55120DDNM30,55120DDNM,55120,ALERCE,0,0,0,0,3,3,2022,0,0,1,1,3,3,0,3,14,0.1429,0.2000,0.1,1,7,21,264,37,33,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,1,2,41,4,5,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0
3,2022-03-24,55120DDNM24,55120DDNM,55120,ALERCE,0,0,0,0,4,3,2022,0,0,1,1,2,3,0,3,14,0.1429,0.1333,0.1,2,8,22,265,48,33,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,1,3,3,7,4,1,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0
4,2022-03-24,55120DDNM31,55120DDNM,55120,ALERCE,0,0,0,0,4,3,2022,0,0,1,1,2,3,0,3,14,0.1429,0.1333,0.1,2,8,22,265,48,33,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,17,5,2,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0


#### Add *Company* related features

In [10]:
conn = connect()
print("#### Adding Company related info")

company_related_features.add_company_last_xdays_sales(conn, 7)
print("Ok feature 1")
company_related_features.add_company_last_xdays_sales(conn, 14)
print("Ok feature 2")
company_related_features.add_company_last_xdays_sales(conn, 30)
print("Ok feature 3")
company_related_features.add_company_historic_sales(conn)
print("Ok feature 4")
company_related_features.add_company_historic_sales_same_day_of_the_week(conn)
print("Ok feature 5")
company_related_features.add_company_historic_sales_same_month(conn)
print("Ok feature 6")
company_related_features.add_company_cumulative_sales_in_the_week(conn)
print("Ok feature 7")
company_related_features.add_company_cumulative_sales_in_the_month(conn)
print("Ok feature 8")
company_related_features.add_company_cumulative_sales_in_the_year(conn)
print("Ok feature 9")
company_related_features.add_company_last_xdays_mean_sales(conn, 7)
print("Ok feature 10")
company_related_features.add_company_last_xdays_mean_sales(conn, 14)
print("Ok feature 11")
company_related_features.add_company_last_xdays_mean_sales(conn, 30)
print("Ok feature 12")

data_holder.set_data(get_datatable_to_df(conn, 'df_sale'))

conn.close()
data_info.preview()

#### Adding Company related info


AttributeError: 'psycopg2.extensions.connection' object has no attribute 'index'

Saving the processed data

In [ ]:
data_holder.get_data().to_csv('../data/temp/processed_and_filled_data.csv')